In [30]:
%load_ext autoreload
%autoreload 2
import os
from functions import *
from functions_unittests import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Check unittests. Equivalent of: unittest.main(argv=[''], verbosity=2, exit=False)
df = get_unittest_dataframe()
display_unittest(df)

CheckStatusNewNameTest
	[OK] Test case where the company exists, name changed and phone changed
	[OK] Test case where the company exists, name changed and url changed
	[OK] Test case where the company exists, name changed and url/phone fields still None
	[OK] Test case where the company exists, only the name has changed
	[OK] Test case where the database is empty
	[OK] Test case where empty finnhub DataFrame
	[OK] Test case where there is a totally new company
KeepNewNamesTest
	[OK] Test case where the company exists, name changed and phone changed
	[OK] Test case where the company exists, name changed and url changed
	[OK] Test case where the company exists, name changed and url/phone fields still None
	[OK] Test case where the company exists, only the name has changed
	[OK] Test case where the database is empty
	[OK] Test case where empty finnhub DataFrame
	[OK] Test case where the company exists
	[OK] Test case where there is a totally new company
------------------------
Ran 15 tes

In [32]:
    # Run only the tests in the specified classe
    test_classes_to_run = [CheckStatusNewNameTest, KeepNewNamesTest]

    loader = unittest.TestLoader()
    suites_list = []
    for test_class in test_classes_to_run:
        suite = loader.loadTestsFromTestCase(test_class)
        suites_list.append(suite)

    # launch the tests
    big_suite = unittest.TestSuite(suites_list)
    runner = unittest.TextTestRunner(stream=sys.stdout, verbosity=2)
    results = runner.run(big_suite)

test_company_exists_name_and_phone_changed (functions_unittests.CheckStatusNewNameTest)
Test case where the company exists, name changed and phone changed ... ok
test_company_exists_name_and_url_changed (functions_unittests.CheckStatusNewNameTest)
Test case where the company exists, name changed and url changed ... ok
test_company_exists_name_and_url_none (functions_unittests.CheckStatusNewNameTest)
Test case where the company exists, name changed and url/phone fields still None ... ok
test_company_exists_only_name_has_changed (functions_unittests.CheckStatusNewNameTest)
Test case where the company exists, only the name has changed ... ok
test_empty_database (functions_unittests.CheckStatusNewNameTest)
Test case where the database is empty ... ok
test_no_company_in_finnhub (functions_unittests.CheckStatusNewNameTest)
Test case where empty finnhub DataFrame ... ok
test_totally_new_company (functions_unittests.CheckStatusNewNameTest)
Test case where there is a totally new company ... ok


In [33]:
# read global connection passwords from bash environment
username = os.environ["POSTGRES_USER"]
password = os.environ["POSTGRES_PASSWORD"]
host = os.environ["HOST"]
bd_name = os.environ["BD_NAME"]

In [34]:
# Read and clean company files
mypath = '../data/'
df_company_profile2 = read_csv_files(mypath)

Number of rows loaded:  120588
Number of rows kept after ticker dropna:  71200
Number of rows kept after drop comp name duplicates:  33496


In [5]:
# Connect Database
session, engine = data_base_connection(username, password, host, bd_name)

In [6]:
# Load data from table
df_merged_base = read_data_base(session)

In [7]:
# Check the data to keep regarding the name
df_company_new_name = keep_new_names(df_company_profile2, df_merged_base)

Number of rows kept after check names already in table:  0


In [8]:
# Check if unknow name is new company or a name changed
df_comp_to_add, df_comp_names_changed = check_status_new_name(df_company_new_name, df_merged_base)

Execution time 0.00 secondes
Number of line(s) treated : 0


In [9]:
# Check the data to keep regarding the contact changes
df_company_new_contacts = keep_new_contacts(df_company_profile2, df_merged_base)

Number of rows kept after check contacts already in table:  0


In [10]:
# Check if unknow name is new company or a contact changed
df_comp_to_add, df_comp_contacts_changed = check_status_new_name(df_company_new_contacts, df_merged_base)

Execution time 0.00 secondes
Number of line(s) treated : 0


In [11]:
# Create new id for new companies
df_comp_to_add = set_company_id(df_comp_to_add, df_merged_base)

In [12]:
# Format the data to insert
df_company, df_comp_description, df_comp_contact, df_comp_ipo = format_data(df_comp_to_add, df_comp_names_changed)

In [13]:
# Insert in database
dtype_res = {"id_company": Integer}
save_table(df_company, dtype_res, "company", engine)

dtype_res = {"date_ipo": String, "id_company": Integer}
save_table(df_comp_ipo, dtype_res, "companyipo", engine)

dtype_res = {"date_description": String, "name": String, "id_company": Integer}
save_table(df_comp_description, dtype_res, "companydescription", engine)

dtype_res = {"date_contact": String,
             "weburl": String,
             "logo": String,
             "phone": String,
             "id_company": Integer}
save_table(df_comp_contact, dtype_res, "companycontact", engine)